# US500 Reversion Strategy (15 min)
Vyhodnocení win rate > 80 % na reálných datech SPY přes 3bar lookahead

In [ ]:
!pip install yfinance ta pandas numpy matplotlib
import yfinance as yf
import pandas as pd
import numpy as np
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


In [ ]:
# 2) Stáhni SPY 15m data za posledních 6 měsíců
end   = datetime.now()
start = end - timedelta(days=180)
df    = yf.download('SPY', start=start, end=end, interval='15m').dropna()
df    = df.reset_index().set_index('Datetime')


In [ ]:
# 3) Indikátory
bb     = BollingerBands(close=df['Close'], window=20, window_dev=2)
df['BB_upper'] = bb.bollinger_hband()
df['BB_lower'] = bb.bollinger_lband()
df['RSI14']    = RSIIndicator(close=df['Close'], window=14).rsi()
hl  = df['High'] - df['Low']
hc  = (df['High'] - df['Close'].shift()).abs()
lc  = (df['Low']  - df['Close'].shift()).abs()
tr  = pd.concat([hl, hc, lc], axis=1).max(axis=1)
df['ATR14']    = tr.rolling(14).mean()
df['VolMA20']  = df['Volume'].rolling(20).mean()
df['OBV']      = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()


In [ ]:
# 4) Generuj signály
long_cond = (
    (df['Close'] < df['BB_lower']) &
    (df['RSI14']  < 30)          &
    (df['Volume'] > df['VolMA20']*1.2) &
    (df['OBV']    > df['OBV'].shift())
)
short_cond = (
    (df['Close'] > df['BB_upper']) &
    (df['RSI14']  > 70)          &
    (df['Volume'] > df['VolMA20']*1.2) &
    (df['OBV']    < df['OBV'].shift())
)
df['signal'] = 0
df.loc[long_cond,  'signal'] =  1
df.loc[short_cond, 'signal'] = -1
entries = df[df['signal']!=0].copy()


In [ ]:
# 5) Vyhodnocení win rate (0,5×ATR TP vs 0,25×ATR SL do 3 barů)
results = []
for idx, row in entries.iterrows():
    sig         = row['signal']
    price       = row['Close']
    atr         = row['ATR14']
    future = df['Close'].loc[idx:].iloc[1:4]
    if sig==1:
        tp = any(future >= price + 0.5*atr)
        sl = any(future <= price - 0.25*atr)
    else:
        tp = any(future <= price - 0.5*atr)
        sl = any(future >= price + 0.25*atr)
    results.append(1 if tp and not sl else 0)
trades   = len(results)
wins     = sum(results)
win_rate = wins/trades*100 if trades else 0
print(f'Trades: {trades}, Wins: {wins}, Win rate: {win_rate:.2f}%')


In [ ]:
# 6) Equity křivka
equity = 100 + np.cumsum(results)
plt.figure(figsize=(8,4))
plt.plot(equity)
plt.title(f'Win rate: {win_rate:.2f}% — Trades: {trades}')
plt.xlabel('Trade #')
plt.ylabel('Equity')
plt.grid(True)
plt.show()
